In [1]:
!pip install datasets
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 30.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 68.8 MB/s eta 0:00:00


In [3]:
pip install accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.19.0
    Uninstalling accelerate-0.19.0:
      Successfully uninstalled accelerate-0.19.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
# !wget https://raw.githubusercontent.com/phatjkk/data/main/LLM/train.json
# !wget https://raw.githubusercontent.com/phatjkk/data/main/LLM/dev.json

--2023-07-11 14:09:37--  https://raw.githubusercontent.com/phatjkk/data/main/LLM/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12012832 (11M) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]  11.46M  --.-KB/s    in 0.1s    

2023-07-11 14:09:37 (118 MB/s) - ‘train.json’ saved [12012832/12012832]

--2023-07-11 14:09:37--  https://raw.githubusercontent.com/phatjkk/data/main/LLM/dev.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2260119 (2.2M) [text/plain]
Saving to: ‘dev.json’



In [1]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

# config = PeftConfig.from_pretrained("phatjk/bloomz-lora-vi-QA-NLLB")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1")
# model.enable_input_require_grads()
# model = PeftModel.from_pretrained(model, "phatjk/bloomz-lora-vi-QA-NLLB",is_trainable=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")

In [5]:
# import pandas as pd
# import json

# def json_to_dataframe(file):
#     f = open ( file , "r")
#     data = json.loads(f.read())               #loading the json file.
#     iid = []
#     tit = []                                  #Creating empty lists to store values.
#     con = []
#     Que = []
#     Ans_st = []
#     Txt = []

#     for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
#         title = data['data'][i]['title']
#         for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
#             context = data['data'][i]['paragraphs'][p]['context']
#             for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
#                 question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
#                 Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
#                 for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags.
#                     ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
#                     text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']

#                     tit.append(title)
#                     con.append(context)
#                     Que.append(question)                    # Appending values to lists
#                     iid.append(Id)
#                     Ans_st.append(ans_start)
#                     Txt.append(text)

#     new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
#     new_df.Id = iid
#     new_df.title = tit           #intializing list values to the DataFrame.
#     new_df.context = con
#     new_df.question = Que
#     new_df.ans_start = Ans_st
#     new_df.text = Txt

#     final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
#     return final_df

In [6]:
# df_train = json_to_dataframe("train.json")
# df_test = json_to_dataframe("dev.json")

In [7]:
# from datasets import Dataset
# ds_train = Dataset.from_pandas(df_train)
# ds_test = Dataset.from_pandas(df_test)

In [8]:
# ds_train

Dataset({
    features: ['Id', 'title', 'context', 'question', 'ans_start', 'text', '__index_level_0__'],
    num_rows: 19240
})

In [9]:
# # Kết hợp câu hỏi và câu trả lời liền kề nhau để model học nội dung
# # Mục đích là để khi đưa 1 đoạn câu hỏi, model sẽ tự nhiểu và gen ra câu trả lời
# # phía sau
# def merge_columns(example):
#     example["prediction"] =  "Hỏi:"+ example["question"] + '''
#     Đáp: ''' + str(example["text"])
#     return example

In [10]:
# data_train = ds_train.map(merge_columns)
# data_test = ds_test.map(merge_columns)

Map:   0%|          | 0/19240 [00:00<?, ? examples/s]

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

In [11]:
# data_train[0]

{'Id': 'uit_000001',
 'title': 'Phạm Văn Đồng',
 'context': 'Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).',
 'question': 'Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?',
 'ans_start': 507,
 'text': 'Lâm Bá Kiệt',
 '__index_level_0__': 0,
 'prediction': 'Hỏi:Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?\n    Đáp: Lâm Bá Kiệt'}

In [12]:
# # Tokenizer/Véc tơ hóa văn bản (Chuyển văn bản thành số để training)
# def tokeni(example):
#     example["prediction"] = tokenizer(example["prediction"], return_tensors='pt', padding=True)['input_ids']
#     return example

In [13]:
# # Map function trên vào dataset
# tokenized_data_train = data_train.map(
#     tokeni,
#     batched=True
# )
# tokenized_data_test = data_test.map(
#     tokeni,
#     batched=True
# )

Map:   0%|          | 0/19240 [00:00<?, ? examples/s]

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

In [3]:
from datasets import load_dataset
data_link = "https://huggingface.co/datasets/phatjk/NLLB_viquad_token_QA/resolve/main/data-00000-of-00001.arrow"
data_files = {"train": data_link}
ds = load_dataset("arrow", data_files=data_files, split="train")

Found cached dataset arrow (/home/.cache/huggingface/datasets/arrow/default-c997381d28e9ee63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137)


In [16]:
# tokenized_data_train

Dataset({
    features: ['Id', 'title', 'context', 'question', 'ans_start', 'text', '__index_level_0__', 'prediction'],
    num_rows: 19240
})

In [7]:
ds

Dataset({
    features: ['prediction'],
    num_rows: 71000
})

In [4]:
# Hàm tính số lượng parameters có thể train được trên model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
# Tạo LoraConfig để áp dụng Lora vào việc training mô hình
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=64, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 9437184 || all params: 1074751488 || trainable%: 0.878080570752371


In [32]:
from datasets import concatenate_datasets
dataset_cc = concatenate_datasets([tokenized_data_train, ds])
shuffled_dataset = dataset_cc.shuffle(seed=42)
shuffled_dataset_prediction = shuffled_dataset.select_columns(["prediction"])
shuffled_dataset_prediction.save_to_disk("test.hf")

In [7]:
# Training model bằng hàm trainer
import transformers
trainer = transformers.Trainer(
    model=model,
    train_dataset=ds["prediction"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        num_train_epochs=1,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        save_total_limit=5,
        learning_rate=2e-4,
        fp16=True,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.640300
1000,2.383000
1500,2.404100
2000,2.343300
2500,2.353200
3000,2.304700
3500,2.313200
4000,2.276100
4500,2.285400
5000,2.283300


TrainOutput(global_step=35500, training_loss=2.177375319789833, metrics={'train_runtime': 12036.7584, 'train_samples_per_second': 5.899, 'train_steps_per_second': 2.949, 'total_flos': 2.3314640859839693e+17, 'train_loss': 2.177375319789833, 'epoch': 1.0})

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
model_name = "bloomz-lora-vi-QA-NLLB-viquad"
HUGGING_FACE_USER_NAME="phatjk"
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/phatjk/bloomz-lora-vi-QA-NLLB-viquad/commit/dc791b1671d76379b75c469b43e34ee88a369d42', commit_message='Upload model', commit_description='', oid='dc791b1671d76379b75c469b43e34ee88a369d42', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
question3 = "Bạn là ai?"
prompt = "Hỏi:"+ question3 + '''
    Đáp:'''
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=100,
                             no_repeat_ngram_size=2,
                             num_beams=3,
                             num_return_sequences=3)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Bạn là ai?
    Đáp: Là một mô hình ngôn ngữ AI, tôi không có khả năng thực hiện các hoạt động cá nhân như đi du lịch, mua sắm hoặc thậm chí là giao tiếp với người khác Tuy nhiên tôi có thể cung cấp cho bạn một số thông tin về các ứng dụng và dịch vụ có sẵn trên điện thoại thông minh và máy tính bảng của tôi Dưới đây là một loạt các tính năng của một vài trong số đó 1 Tìm kiếm và tìm kiếm Bạn có một danh sách các địa điểm du
Hỏi:Bạn là ai?
    Đáp: Là một mô hình ngôn ngữ AI, tôi không có khả năng thực hiện các hoạt động cá nhân như đi du lịch, mua sắm hoặc thậm chí là giao tiếp với người khác Tuy nhiên tôi có thể cung cấp cho bạn một số thông tin về các ứng dụng và dịch vụ có sẵn trên điện thoại thông minh và máy tính bảng của tôi Dưới đây là một loạt các tính năng của một vài trong số đó 1 Tìm kiếm và tìm kiếm Bạn có một danh sách các địa điểm bạn
Hỏi:Bạn là ai?
    Đáp: Là một mô hình ngôn ngữ AI, tôi không có khả năng thực hiện các hoạt động cá nhân như đi du lịch, mua sắm hoặc 

In [11]:
question2 = "Viết bài tả con mèo."
prompt = "Hỏi:"+ question2 + '''
    Đáp:'''
prompt
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=100,
                             no_repeat_ngram_size=2,
                             num_beams=3,
                             num_return_sequences=1)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Viết bài tả con mèo.
    Đáp: Con mèo là một trong những động vật có vú phổ biến nhất trên thế giới Nó được biết đến với vẻ đẹp của nó và sức mạnh của chúng Nó có thể được tìm thấy ở khắp mọi nơi trên Trái đất từ Bắc Cực đến Nam Cực và từ châu Phi đến châu Á Nó thường được gọi là mèo vì nó có lông dài và lông ngắn Chúng có một số đặc điểm khác nhau như màu sắc và hình dạng của lông và có nhiều loại lông như lông màu đen và màu trắng Chúng cũng


In [9]:
pt_save_directory = "./bloomz-lora-vi-QA-NLLB-visquad/"
trainer.model.save_pretrained(pt_save_directory)
tokenizer.save_pretrained(pt_save_directory)

('./bloomz-lora-vi-QA-NLLB-visquad/tokenizer_config.json',
 './bloomz-lora-vi-QA-NLLB-visquad/special_tokens_map.json',
 './bloomz-lora-vi-QA-NLLB-visquad/tokenizer.json')